In [7]:
import pandas as pd
import lightgbm as lgb
import time
path='/home/student303/oyx/almama/'

#Read Data
data = pd.read_hdf(path+ 'somefeature/base_feature_all_0.h5', key = 'a')
print('Read Data Finish.')

#Define featurelist
droplist=['instance_id',
          'item_category_list',
          'item_property_list',
          'user_id', 
          'context_id',
          'context_timestamp',
          'predict_category_property',
          'is_trade',
          'day', 'min', 'second',
          'user_nextclick_interval',
         # 'user_nextclick_interval_mean'
         ]
featurelist=[i for i in data.columns if i not in droplist]
cat_feature=[
             'item_id',
             'item_brand_id',
             'item_city_id',
             'user_gender_id',
             'user_occupation_id',
             'context_page_id',
             'shop_id',
             'item_category_hash',
             'hour',
            ]

Read Data Finish.


In [10]:
featurelist

['item_id',
 'item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_gender_id',
 'user_age_level',
 'user_occupation_id',
 'user_star_level',
 'context_page_id',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_star_level',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'item_category_hash',
 'hour']

In [11]:
%%time
dev=1
print('LGBM Training, Feature dim : ',len(featurelist))
params = {
    'task': 'train',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth':4,
    #'num_leaves': 6,
    #'min_data_in_leaf':1,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 1.0,
    'verbose': 0,
}
if dev:
    d_train=lgb.Dataset(data.loc[data.day < 24][featurelist], label=data.loc[data.day < 24]['is_trade'],categorical_feature=cat_feature)
    d_valid=lgb.Dataset(data.loc[data.day == 24][featurelist], label=data.loc[data.day == 24]['is_trade'],categorical_feature=cat_feature)
    model = lgb.train(params, train_set=d_train, num_boost_round=600, valid_sets=[d_train,d_valid], \
                  early_stopping_rounds=50, verbose_eval=10)
    offlinelogloss=model.best_score['valid_1']['binary_logloss']
    bst_iteration=model.best_iteration
else:
    d_train=lgb.Dataset(data.loc[data.day <= 24][featurelist], label=data.loc[data.day <= 24]['is_trade'],categorical_feature=cat_feature)    
    model = lgb.train(params, train_set=d_train, num_boost_round=bst_iteration, valid_sets=[d_train], \
                   verbose_eval=10)
print('LGBM Training Done')

LGBM Training, Feature dim :  21
Train until valid scores didn't improve in 50 rounds.
[10]	training's binary_logloss: 0.239135	valid_1's binary_logloss: 0.235815
[20]	training's binary_logloss: 0.130002	valid_1's binary_logloss: 0.124579
[30]	training's binary_logloss: 0.100194	valid_1's binary_logloss: 0.0934147
[40]	training's binary_logloss: 0.0927128	valid_1's binary_logloss: 0.0853352
[50]	training's binary_logloss: 0.0907958	valid_1's binary_logloss: 0.0833254
[60]	training's binary_logloss: 0.0900719	valid_1's binary_logloss: 0.0827442
[70]	training's binary_logloss: 0.0896333	valid_1's binary_logloss: 0.0824794
[80]	training's binary_logloss: 0.089291	valid_1's binary_logloss: 0.0823535
[90]	training's binary_logloss: 0.0890182	valid_1's binary_logloss: 0.0822759
[100]	training's binary_logloss: 0.088751	valid_1's binary_logloss: 0.0822473
[110]	training's binary_logloss: 0.0885332	valid_1's binary_logloss: 0.0821977
[120]	training's binary_logloss: 0.0883472	valid_1's binary_

In [ ]:
if not dev:
    prob = model.predict(data.loc[data.day > 24][featurelist], num_iteration=bst_iteration)
    output=pd.DataFrame({'instance_id':data.loc[data.day > 24].instance_id.values ,'predicted_score':prob})  
    print('Predicted prob mean is ',prob.mean())

    output_name='Sub_'+time.strftime('%m%d',time.localtime(time.time()))+'_%.5f'%(offlinelogloss)
    output.to_csv(path+'output/'+output_name+'.txt', sep=" ", index=False, line_terminator='\n')

    print('Output success, name : ',output_name)
    print(output.head())